# File Watcher Demo

This notebook demonstrates how to use the syft-client file watcher for automatic synchronization.

## Setup

First, let's import and login to syft-client:

In [ ]:
import syft_client as sc

# Login with your email
client = sc.login("your_email@gmail.com")

## Starting the Watcher

The watcher monitors your SyftBox directory for changes and automatically syncs them with your peers:

In [ ]:
# Start the watcher (runs as a persistent server process)
client.start_watcher()

## Check Watcher Status

You can check if the watcher is running and get its status:

In [ ]:
# Check if watcher is running
client.watcher.is_running()

In [ ]:
# Get detailed status
client.watcher.status()

## How It Works

1. **File Monitoring**: The watcher monitors `~/SyftBox/datasites/{your_email}/` for changes
2. **Automatic Sending**: When files are created or modified, they're automatically sent to all peers
3. **Bidirectional Sync**: The watcher also polls your inbox for messages from peers
4. **Echo Prevention**: Smart detection prevents sync loops between peers
5. **Persistent Process**: The watcher runs as a separate process that survives Python restarts

## Test the Watcher

Try creating a file in your SyftBox directory and watch it sync automatically:

In [ ]:
from pathlib import Path
import datetime

# Get your SyftBox directory
syftbox_dir = Path.home() / "SyftBox" / "datasites" / client.email
syftbox_dir.mkdir(parents=True, exist_ok=True)

# Create a test file
test_file = syftbox_dir / f"test_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
test_file.write_text(f"Hello from the watcher!\nCreated at {datetime.datetime.now()}")

print(f"Created test file: {test_file}")
print("\nThe watcher should automatically send this to your peers!")

## View Sync History

You can check the sync history for any file:

In [ ]:
# Get sync history for a file
history = client.watcher.get_history(str(test_file), limit=5)
if history:
    for record in history:
        print(f"- {record['direction']} to/from {record['peer']} via {record['transport']} at {record['timestamp']}")
else:
    print("No sync history yet (file might still be processing)")

## List All Active Watchers

See all watchers running on your system:

In [ ]:
# List all active watchers
watchers = client.watcher.list_all()
for w in watchers:
    print(f"- {w['email']} at {w['server_url']}")

## Stopping the Watcher

When you're done, stop the watcher:

In [ ]:
# Stop the watcher
client.watcher.stop()

## Advanced: Environment Variables

You can configure the watcher behavior with environment variables:

- `SYFT_SYNC_ECHO_THRESHOLD`: Seconds to wait before considering a file change as new (default: 60)
- `SYFT_INBOX_POLL_INTERVAL`: Seconds between inbox checks (default: 30)

In [ ]:
import os

# Set custom echo threshold (5 seconds for testing)
os.environ['SYFT_SYNC_ECHO_THRESHOLD'] = '5'

# Set faster inbox polling (10 seconds)
os.environ['SYFT_INBOX_POLL_INTERVAL'] = '10'

# Start watcher with custom settings
client.start_watcher()

## Notes

- The watcher runs as a separate syft-serve process
- It continues running even after you close this notebook
- Only one watcher can run per email at a time
- The watcher automatically skips hidden files and temporary files
- Echo prevention ensures files don't sync back and forth infinitely